In [4]:
import urllib as ul
import math
import heapq as hq

# División en sectores de los puntos de almacen y los puntos de entrega mas cerca

In [5]:
def readAdjl(filename):
  with ul.request.urlopen(filename) as f:
    L = []
    for line in f:
        L.append(list(map(int, (line.decode("utf-8").strip().split(",")))))
  return L

def getIndex(coord, size):
    return coord[0] * size + coord[1]

def getCoords(index, size):
  x = index // size
  y = index % size
  return (x, y)

def getDistance(x1, y1, x2, y2):
    return math.sqrt( (x1 - x2) ** 2 + (y1 - y2) **2 )

def readFileWithCoords(filename, size):
  arr = []
  with ul.request.urlopen(filename) as f:
    for line in f:
      coord = list(map(int, (line.decode("utf-8").strip().split(","))))
      arr.append(getIndex(coord, size))
  return arr

In [6]:
G = readAdjl("https://raw.githubusercontent.com/RodriCalle/cc41_tf_201915889_201910127_201917028_201718169_20141a449/master/adjancencyList.txt")

urlAlmacenes = "https://raw.githubusercontent.com/RodriCalle/cc41_tf_201915889_201910127_201917028_201718169_20141a449/master/datasets/almacenes.csv"
urlEntregas = "https://raw.githubusercontent.com/RodriCalle/cc41_tf_201915889_201910127_201917028_201718169_20141a449/master/datasets/puntos_entrega.csv"

index_almacenes = readFileWithCoords(urlAlmacenes, 1000)
index_entregas = readFileWithCoords(urlEntregas, 1000)

In [7]:
# Verificando distancias para asignar el punto de entrega al amacen
def nearestPoints(pointEntrega):
    xE, yE = getCoords(pointEntrega, 1000)
    nearestAlmacen = index_almacenes[0]
    for pointAlmacen in index_almacenes:
        xA, yA = getCoords(pointAlmacen, 1000)
        xaux, yaux = getCoords(nearestAlmacen, 1000)
        if getDistance(xA,yA,xE,yE) < getDistance(xaux,yaux,xE,yE):
            nearestAlmacen = pointAlmacen
    return nearestAlmacen

# Creacion de diccionario con clave (Punto de almacen) y valor (Puntos de entregas mas cercano)
dictPoints = {}

for pointEntrega in index_entregas:
    nearestAlmacen = nearestPoints(pointEntrega)
    if not nearestAlmacen in dictPoints:
        dictPoints[nearestAlmacen] = [pointEntrega]
    else:
        dictPoints[nearestAlmacen].append(pointEntrega)

# Mostrando los resultados del diccionarios, con su clave y valor
counterEntregas = 0
for i, pointAlmacen in enumerate(index_almacenes):
    if not pointAlmacen in dictPoints:
        dictPoints[pointAlmacen] = []
    print(f"Almacen {i}, N° {pointAlmacen}: {dictPoints[pointAlmacen]}")
    counterEntregas += len(dictPoints[pointAlmacen])

print(f"\nCantidad total de puntos de entrega: {counterEntregas}")

Almacen 0, N° 496838: [477817, 484874, 555837, 488762, 502783, 552813, 483866, 553799, 480855, 521778, 442855, 435867, 553853, 467832, 490866, 524770, 455884, 501783, 479793, 442895, 537780, 539781, 541852, 550859, 539863, 457858, 451856, 472775, 554808, 513801, 550783, 529809, 501807, 512811, 467812, 527833, 517770, 508774, 527784, 513814, 473883, 486768, 521822, 528793, 513871, 540796, 500849, 513838, 484761, 490823, 539789, 513796, 449862, 553848, 487843, 537859]
Almacen 1, N° 15628: [51575, 60647, 63591, 17704, 47737, 65628, 2598, 15690, 58743, 25744, 3733, 19668, 12637, 67712, 42735, 32713, 73688, 31694, 48573, 10668, 5719, 6604, 34705, 57732, 19659, 66726, 80655, 60656, 61702, 58662, 8687, 36618, 23748, 41722, 54701, 78678, 12603, 38707, 54600, 68699, 52580, 33568, 51620, 60708, 17624, 40718, 46604, 13765, 48611, 59700, 41639, 43757, 13703, 45619, 3574, 68706, 15748, 10664, 53611, 29615, 23746, 18643, 64685, 52692, 44565]
Almacen 2, N° 877982: [890995, 940994, 863991, 862990, 902

In [8]:
def newDijkstra(G, s):
  visited = {}
  path = {}
  cost = {}

  for k in G.keys():
    visited[k] = False
    path[k] = None
    cost[k] = math.inf

  cost[s] = 0
  queue = [(s, 0)]
  while queue:
    u, g_u = hq.heappop(queue)
    if not visited[u]:
      visited[u] = True
      for v in G[u]:
        f = g_u + 1
        if f < cost[v]:
          cost[v] = f
          path[v] = u
          hq.heappush(queue, (v, f))
  return path

In [14]:
def generateAdjl(puntosEntrega, init):
    adjacency_list = {}
    maxX, maxY = getCoords(init,1000)
    minX, minY = getCoords(init,1000)

    for entregas in puntosEntrega:
        entX, entY = getCoords(entregas, 1000)
        if entX > maxX:
            maxX = entX
        elif entX < minX:
            minX = entX
        if entY > maxY:
            maxY = entY
        elif entY < minY:
            minY = entY
    
    for x in range(minX, maxX + 1):
        for y in range(minY, maxY + 1):
            adjacency_list[getIndex((x,y),1000)] = []

    for point in adjacency_list:
        x, y = getCoords(point, 1000)
        left_point = getIndex((x - 1, y), 1000)
        right_point = getIndex((x + 1, y), 1000)
        up_point = getIndex((x, y + 1), 1000)
        down_point = getIndex((x, y - 1), 1000)

        if left_point in adjacency_list:
            adjacency_list[point].append(left_point)
        if right_point in adjacency_list:
            adjacency_list[point].append(right_point)
        if up_point in adjacency_list:
            adjacency_list[point].append(up_point)
        if down_point in adjacency_list:
            adjacency_list[point].append(down_point)

    return adjacency_list


def pathsToNode(path, entrega, init):
  tr = []
  change_x = 0
  change_y = 0
  while entrega != None:
    if path[entrega] == -1: break
    parent = path[entrega]
    if parent != None:
      entregaX, entregaY = getCoords(entrega, 1000)
      parentX, parentY = getCoords(parent, 1000)
      aux_change_x = change_x
      aux_change_y = change_y
      if entregaX != parentX:
        change_x = 1
        if change_x != aux_change_x:
          change_y = 0
          tr.append(entrega)
      if entregaY != parentY:
        change_y = 1
        if change_y != aux_change_y:
          change_x = 0
          tr.append(entrega)
    entrega = parent
  
  tr.append(init)
  tr.reverse()
  return tr

In [15]:
for index, p_a in enumerate(index_almacenes):
    adjList = generateAdjl(dictPoints[p_a], p_a)
    path = newDijkstra(adjList, p_a)
    arrCaminos = []
    for p_e in dictPoints[p_a]:
         arrCaminos.append(PathsToNode(path, p_e, p_a))
    print(f"Almacen {index} ({p_a}) : {arrCaminos}")

Almacen 0 (496838) : [[496838, 435838, 435817, 477817], [496838, 484838, 484874], [496838, 496837, 555837], [496838, 435838, 435762, 488762], [496838, 435838, 435783, 502783], [496838, 435838, 435813, 552813], [496838, 483838, 483866], [496838, 435838, 435799, 553799], [496838, 480838, 480855], [496838, 435838, 435778, 521778], [496838, 442838, 442855], [496838, 435838, 435867], [496838, 496853, 553853], [496838, 435838, 435832, 467832], [496838, 490838, 490866], [496838, 435838, 435770, 524770], [496838, 455838, 455884], [496838, 435838, 435783, 501783], [496838, 435838, 435793, 479793], [496838, 442838, 442895], [496838, 435838, 435780, 537780], [496838, 435838, 435781, 539781], [496838, 496852, 541852], [496838, 496859, 550859], [496838, 496863, 539863], [496838, 457838, 457858], [496838, 451838, 451856], [496838, 435838, 435775, 472775], [496838, 435838, 435808, 554808], [496838, 435838, 435801, 513801], [496838, 435838, 435783, 550783], [496838, 435838, 435809, 529809], [496838, 4

# Division de regiones

Division de regiones por integrantes, la funcion getParameters brinda como resultado dos numeros, el inicio y fin que indica la cantidad de almacenes que se le otorga al integrante.

In [12]:
def getParameters(number):
    qPerPerson = len(index_almacenes)/4
    start = qPerPerson*number
    end = start + qPerPerson
    return start, end

# Algoritmo de cada integrante

In [17]:
# Rodrigo Calle Galdos - u201915889
def newPrim(G,p_a):
  visited = {}
  path = {}
  cost = {}
  for key in G.keys():
    visited[key] = False
    path[key] = None
    cost[key] = math.inf
  cost[p_a] = 0
  q = [(0, p_a)]
  while q:
    _, u = hq.heappop(q)
    if not visited[u]:
      visited[u] = True
      for v in G[u]:
        if not visited[v] and 1 < cost[v]:
          cost[v] = 1
          path[v] = u
          hq.heappush(q, (1, v))

  return path

In [ ]:
# Juan de Dios Quiroz Rodriguez - u201910127

In [ ]:
# Carlos Leon Rupay - u201917028

In [ ]:
# Adrian Esqueiros Cabrera - u201718169

# Aplicación de los algoritmos personales

In [18]:
# Rodrigo Calle Galdos - u201915889

start, end = getParameters(0)
print("Almacen de inicio:",start)
print("Almacen de fin:",end)

for i, point in enumerate(index_almacenes):
    if i <= start or i > end: continue
    adjl = generateAdjl(dictPoints[point], point)
    path = newPrim(adjl, point)
    arr = []
    for pointEntrega in dictPoints[point]:
         arr.append(pathsToNode(path, pointEntrega, point))
    print(f"Almacen {i} ({p_a}) : {arr}")
    break

Almacen de inicio: 0.0
Almacen de fin: 25.0
Almacen 1 (609210) : [[15628, 2628, 2575, 51575], [15628, 2628, 2647, 60647], [15628, 2628, 2591, 63591], [15628, 2628, 2704, 17704], [15628, 2628, 2737, 47737], [15628, 65628], [15628, 2628, 2598], [15628, 2628, 2690, 15690], [15628, 2628, 2743, 58743], [15628, 2628, 2744, 25744], [15628, 2628, 2733, 3733], [15628, 2628, 2668, 19668], [15628, 2628, 2637, 12637], [15628, 2628, 2712, 67712], [15628, 2628, 2735, 42735], [15628, 2628, 2713, 32713], [15628, 2628, 2688, 73688], [15628, 2628, 2694, 31694], [15628, 2628, 2573, 48573], [15628, 2628, 2668, 10668], [15628, 2628, 2719, 5719], [15628, 2628, 2604, 6604], [15628, 2628, 2705, 34705], [15628, 2628, 2732, 57732], [15628, 2628, 2659, 19659], [15628, 2628, 2726, 66726], [15628, 2628, 2655, 80655], [15628, 2628, 2656, 60656], [15628, 2628, 2702, 61702], [15628, 2628, 2662, 58662], [15628, 2628, 2687, 8687], [15628, 2628, 2618, 36618], [15628, 2628, 2748, 23748], [15628, 2628, 2722, 41722], [1562

In [ ]:
# Juan de Dios Quiroz Rodriguez - u201910127

start, end = getParameters(1)
print("Almacen de inicio:",start)
print("Almacen de fin:",end)

Almacen de inicio: 25.0
Almacen de fin: 50.0


In [ ]:
# Carlos Leon Rupay - u201917028

start, end = getParameters(2)
print("Almacen de inicio:",start)
print("Almacen de fin:",end)

Almacen de inicio: 50.0
Almacen de fin: 75.0


In [ ]:
# Adrian Esqueiros Cabrera - u201718169

start, end = getParameters(3)
print("Almacen de inicio:",start)
print("Almacen de fin:",end)

Almacen de inicio: 75.0
Almacen de fin: 100.0
